# Aaron's ISR challenge - are amp boundaries visible after ISR?
Implementing 20 pixel edge mask in PTC \
Craig Lage 25-Jan-23

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from lsst.cp.pipe.ptc import PlotPhotonTransferCurveTask, PlotPhotonTransferCurveConfig
%matplotlib inline


In [ ]:
detector=55
expId = 3021120700188
fullButler = Butler("/repo/main", collections=["u/cslage/bps_13144S"])
ptcButler = Butler("/repo/main", collections=["u/cslage/ptc_subregion_full_8C_12feb23"])
ptc = ptcButler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")

fullPtc = fullButler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")
exp = ptcButler.get('raw', detector=detector, exposure=expId, instrument="LSSTCam")
camera = ptcButler.get('camera', instrument='LSSTCam')

In [ ]:
ptc1Butler = Butler("/repo/main", collections=["u/cslage/ptc_subregion_full_1_12feb23"])
ptc1 = ptc1Butler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")

baseButler = Butler("/repo/main", collections=["u/cslage/ptc_subregion_full_base_12feb23"])
basePtc = butler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")

In [ ]:
plotTask = PlotPhotonTransferCurveTask()

In [ ]:
plotResult = plotTask.run(ptc, dummyExposure=[exp], camera=camera)

In [ ]:
len(basePtc.finalVars['C00'])

In [ ]:
ptc.ptcFitType

In [ ]:
#Why is it rejecting all of the points???
plt.scatter(ptc.rawMeans['C00'], ptc.rawVars['C00'])

In [ ]:
#Why is it rejecting all of the points???
plt.scatter(ptc.rawMeans['C00'], ptc.rawVars['C00'], label="2x4")
plt.scatter(ptc1.rawMeans['C00'], ptc1.rawVars['C00'], marker='x', s=200, color='red', label="Baseline")
plt.plot(ptc1.finalMeans['C00'], ptc1.finalModelVars['C00'], color='red', label="Baseline")
plt.xlim(40000,45000)
plt.ylim(22000,27000)
plt.legend()

In [ ]:
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter

x = np.array(ptc.rawMeans['C00'])
y = np.array(ptc.rawVars['C00'])
xx = np.linspace(x.min(),x.max(), 1000)

# interpolate + smooth
itp = interp1d(x,y, kind='linear')
window_size, poly_order = 101, 3
yy_sg = savgol_filter(itp(xx), window_size, poly_order)



In [ ]:
plt.scatter(xx, yy_sg)
index = np.argmax(yy_sg)
ptc_turnoff = xx[index]
plt.plot([ptc_turnoff, ptc_turnoff], [0,50000])
#plt.xlim(80000,92500)
#plt.ylim(40000,44000)

In [ ]:
for amp in exp.getDetector().getAmplifiers():
    print(amp.getName(), ptc.gain[amp.getName()], basePtc.gain[amp.getName()], ptc1.gain[amp.getName()])